In [1]:
import numpy as np
from itertools import *
from sympy.combinatorics.graycode import GrayCode
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute, Aer
from qiskit.tools.visualization import circuit_drawer

C:\Users\Chams\Anaconda2\envs\hackathon\lib\site-packages\qiskit\__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


# Generating partition of G
The following functions generate the partition into a maximal collection of invariant subsets

In [2]:
def fullSymDecomp(n,start,end):
    ''' Returns dictionary with FULLY symmetric group G partition (decimal values) into maximal subsets
        n: total number of qubits
        start: position of the first element of the fully symmetric group (starting from 0)
        end: position of the last element of the fully symmetric group
    '''
    Gpart = {}
    for i in range(end-start+1):
        Gpart[i] = []
        for comb in list(combinations(range(end-start),i)):
            cval = 0
            for el in comb:
                cval += 2**(n-1-(el+start))
            Gpart[i].append(cval)
    return Gpart

def combine(sys):
    ''' Return dictionary with G partition (decimal values) into maximal subsets by combining the fully Symmetric blocks
        sys: list with the sizes of the permutation sub-groups '''
    start = 0
    n = sum(sys)
    Gpart = fullSymDecomp(n,0,sys[0])
    if len(sys) > 1:
        start = sys[0]
        for gsize in sys[1:]:
            end = start+gsize
            oldGpart = Gpart
            Gpart = fullSymDecomp(n,start,end)
            newGpart = {}
            count = 0
            for oldkey in oldGpart:
                for key in Gpart:
                    newGpart[count] = []
                    for el in Gpart[key]:
                        for oldel in oldGpart[oldkey]:
                            newGpart[count].append(el+oldel)
                    count += 1
            start += gsize
            Gpart = newGpart
    return Gpart

def convertGToBinary(Gpart,n):
    ''' Convert the decimal indices used to construct the subsets into binary arrays
        n: total number of qubits '''
    binGpart = {k: [("{0:0"+str(n)+"b}").format(el) for el in v] for k, v in Gpart.items()}
    return binGpart

def verifyPermutation(sys,pi):
    start = 0
    for i in range(len(sys)):
        expsum,actsum = 0,0
        for j in range(start,start+sys[i]):
            expsum += j+1
            actsum += int(pi[j])
        if expsum != actsum:
            raise TypeError('Your input permutation does not belong to the defined group')
        start += sys[i]

# Angles for the controlled rotations
The definition of the angles for the initial and controlled rotations used in the quantum circuit to generate the transversal

In [3]:
def alpha1(T,n):
    ''' Calculation of initial rotation angle
        INPUT T: list with the indices (converted to decimal) of the elements in the transversal
              n: total number of qubits in current fully symmetric block of the circuit
        OUTPUT: rotation angle \alpha_1
    '''
    sum1,sum2 = 0,0
    for el in T:
        if el >= 2**(n-1): sum1 += 1 # as we work with a uniform superposition, the denominators of the coefficents cancel out,
                                     #   so just +|1|^2 = +1
        else: sum2 += 1
    if sum2 == 0:
        if sum1 == 0:
            raise ZeroDivisionError('Found: 0/0')
        else: return np.pi/2
    else:
        return np.arctan(np.sqrt(sum1/sum2))

def ControlledAlpha(T,n,j,control):
    ''' Calculation of controlled \alpha_j
        INPUT T: list of elements of the transversal
              n: total number of qubits in current fully symmetric block of the circuit
              j: target qubit index (starting from 0 up to n-1, reading the arrays from left to right)
              control: control qubits state in decimal form (calculated for the full qubit array,
                       assuming all the rest are zeros, e.g. '01XX' -> '0100' -> control = 4)
    '''
    sum1,sum2 = 0,0
    for el in T:
        if el >= control and el < control + 2**(n-j):
            if (el-control) >= 2**(n-j-1): sum1 += 1
            else: sum2 += 1
    if sum2 == 0:
        if sum1 == 0:
            raise ZeroDivisionError('Found: 0/0') # as we showed, this should never happen
        else: return np.pi/2
    else:
        return np.arctan(np.sqrt(sum1/sum2))

# Quantum subroutines
+ Controlled-n Ry gate implementation
+ convert a control from mix of zeros and ones to full ones, or unconvert full ones to desired control

In [4]:
def nControlledRy(start,target,theta,q,circ):
    ''' Adds to the quantum circuit a rotation around 'y' gate controlled by any number of qubits, i.e. the rotation
            is performed if all the control qubits are 1. The control qubits are defined in block, e.g. "from qubit0 to qubit4" 
        INPUT start: first qubit used as control
              target: qubit over which the Ry gate is applied (therefore, the control is from qubit 'start' to 'target-1')
              theta: desried angle of rotation (in radians)
              q: quantum register with circuit's qubits
              circ: original circuit in where the nC-Ry gate is implemented
        OUTPUT: new circuit with the nC-Ry(theta) gate added
    '''
    theta = theta/2**(target-start+1-3) # redefine theta adapted for the circuit (see report)
    a = GrayCode(target-start)
    gray = list(a.generate_gray(start='0'*(target-start-1)+'1')) # gray code starting at '00..01' instead of '10..00'
                                                                 #  -> will read them from right to left
    ## ------------------------
    ## Implementing the algorithm to generate a controlled-n following a gray code as explained in the report
    ## ------------------------
    prevel = gray[0]
    lm = start
    parity = 1
    circ.cu3(parity*theta,0,0,control_qubit=q[lm],target_qubit=q[target])
    for el in gray[1:]:
        parity *= -1
        val = int(el,2) - int(prevel,2)
        ind = int(np.log2(abs(val)))+start
        if ind > lm:
            lm = ind
            circ.cx(control_qubit=q[ind-1],target_qubit=q[ind])
        else:
            circ.cx(control_qubit=q[ind],target_qubit=q[lm])
        circ.cu3(parity*theta,0,0,control_qubit=q[lm],target_qubit=q[target])
        prevel = el
    return circ

def AdjustControl(control,n,start,target,q,circ):
    ''' Apply X gates (NOT) to the control qubits to convert them to a full ones control (and the opposite process as well)
        INPUT control: decimal value of the desired control obtained as 'CTRLXXXX' -> 'CTRL0000' -> bin-to-dec('CTRL0000')
              n: total number of qubits in current fully symmetric block of the circuit
              start: first qubit used as control
              target: qubit over which the Ry gate is applied
              q: quantum register with circuit's qubits
              circ: original circuit in where the nC-Ry gate is implemented
        OUTPUT new circuit with the adjusted control
    '''
    ctr = ("{0:0"+str(n)+"b}").format(control)
    for i in range(target-start):
        if int(ctr[i]) == 0:
            circ.x(q[i+start])
    return circ

# Transversal quantum generator for the full symmetric group
The following function puts all together to output a circuit that generates a quantum transversal in uniform superposition

In [5]:
def FullSymQSuperposition(T,n,start,q,circ):
    ''' Circuit generator to create a quantum transversal for the FULLY symmetric group
        INPUT T: on demand fully symmetric group transversal
              n: total number of qubits in current fully symmetric block of the circuit
              start: first qubit of the fully symmetric block
              q: quantum register with circuit's qubits
              circ: original circuit in where the nC-Ry gate is implemented
        OUTPUT quantum circuit generating an on-demand fully symmetric group transversal '''
    for target in range(start,start+n):
        if target == start:
            circ.u3(alpha1(T,n)*2,0,0,q[start])
            circ.z(q[start])
        else:
            for ctl in range(2**(target-start)):
                states = [int(v) for v in list(("{0:0"+str(target-start)+"b}").format(ctl))]
                control = int(("{0:0"+str(target-start)+"b}").format(ctl) + '0'*(n-target+start),2) # calculate control's decimal
                                                                                                    # value within subblock
                for el in T:
                    if el >= control and el < control + 2**(n-target+start): # checking whether T element contains control array
                        theta = ControlledAlpha(T,n,target-start,control)
                        circ = AdjustControl(control,n,start,target,q,circ)
                        circ = nControlledRy(start,target,theta,q,circ)
                        circ = AdjustControl(control,n,start,target,q,circ)
                        break
            circ.z(q[target]) # convert Ry rotation into the desired form [[cos\th sin\th],[sin\th -cos\th]]
    return circ

# Interactive solver
+ Introduce your symmetry group. If composed, separate blocks with 'x'. e.g. 3 for S3, and 2x1 for S2xS1
+ Check whether you want to input a permutation (permuteT) or on-demand transversal (onDemandT) and input the corresponding information
+ Choose the number of shots for the circuit simulation
+ Choose the file name (or path/filename) for your output circuits (in '.tex' and '.png' format)

(Some errors are raised by the program if some of the inputs are wrong in some way. In case of facing such error, please skip the traceback part and go directly to the end where you will see: TypeError: "description of what went wrong")

OUTPUT:
+ Transversal and eventually the permuted transversal
+ QASM simulations results given by the number of measurements for each superimposed state
+ Result verifications
+ Drawing of the circuit exported in .png and .tex format

In [6]:
from ipywidgets import widgets
from IPython.display import display,clear_output

class quantumTransversal(object):
    def __init__(self):
        # defaults
        self.permT = False
        self.ondemand = False
        self.sys = [3]
        self.n = sum(self.sys)
        self.pi = '123'
        self.T,self.odT,self.pT = [],[],[]
        self.nshots = 1024
        self.filename = 'output'
    def defineSys(self,symgroup):
        sys = []
        for el in symgroup.split('x'):
            if len(el) > 0:
                sys.append(int(el))
        self.sys = sys
        self.n = sum(sys)
    def permuteT(self,permuteT):
        self.permT = permuteT
    def isOnDemand(self,onDemandT):
        self.ondemand = onDemandT
    def onDemandT(self,onDemandT):
        odT = []
        for el in onDemandT.split(','):
            if len(el) > 0: odT.append(int(el,2))
        self.odT = odT
    def permutation(self,T_permutation):
        if type(T_permutation) == str:
            self.pi = T_permutation
        else: raise TypeError("Wrong input format. Please enter a length 'n' string, such as '132'")
    def shotN(self,NoOfShots):
        if len(NoOfShots) > 0:
            self.nshots = int(NoOfShots)
    def setfilename(self,path_filename):
        self.filename = path_filename
        
    def buildT(self):
        G = combine(self.sys)
        print('G partition into maximal collection of subsets:')
        print(convertGToBinary(G,self.n))
        print('')
        T = []
        for key in G:
            T.append(G[key][0])
        self.T = T
    def applyPermutation(self):
        if len(self.pi) != self.n:
            raise TypeError('Wrong permutator length. Please reenter a length "n" string for the permutator')
        newT = []
        verifyPermutation(self.sys,self.pi)
        for el in self.T:
            oldt = ("{0:0"+str(self.n)+"b}").format(el)
            newt = ''
            for i in self.pi:
                newt += str(oldt[int(i)-1])
            newT.append(int(newt,2))
        self.pT = newT
        
    def run(self,b):
        with out:
            clear_output()
            qT.buildT()
            if self.ondemand: T = self.odT
            else: T = self.T
            print('Transversal')
            print([("{0:0"+str(self.n)+"b}").format(el) for el in T])
            print('')
            if self.permT:
                qT.applyPermutation()
                print('Permuted Transversal')
                print([("{0:0"+str(self.n)+"b}").format(el) for el in self.pT])
                print('')
                T = self.pT
            
            q = QuantumRegister(self.n,'qubit')
            c = ClassicalRegister(self.n,'bit')
            circ = QuantumCircuit(q,c)
            
            if not self.ondemand:
                start = 0
                for i in range(len(self.sys)):
                    ns = self.sys[i]
                    Ts = [int(("{0:0"+str(self.n)+"b}").format(t)[start:start+ns],2) for t in T]
                    circ = FullSymQSuperposition(Ts,ns,start,q,circ)
                    start += ns
            else:
                circ = FullSymQSuperposition(T,self.n,0,q,circ)
            
            print('Generating quantum circuit...')
            print('')
            print('QASM simulation results out of a total of ' + str(self.nshots) + ' shots')
            circ.measure(q,c)
            # Execute the circuit
            job = execute(circ, backend = Aer.get_backend('qasm_simulator'), shots=self.nshots)
            result = job.result()
            reversedOutput = {key[::-1]:result.get_counts(circ)[key] for key in result.get_counts(circ)}
            sortedOutput = {key:reversedOutput[key] for key in sorted(reversedOutput)}
            print(sortedOutput)
            print('')
            
            print('Testing results...')
            fail,count = 0,0
            for key in sortedOutput:
                count += 1
                if int(key,2) not in T:
                    print('Wrong output state :(')
                    fail = 1
                    break
            if fail == 0:
                if count != len(T):
                    print('Wrong output state :(')
                else:
                    print('Perfect match of superimposed states!')

            avgcount = self.nshots/len(T)
            std = 0
            for key in sortedOutput:
                std += abs(sortedOutput[key]-avgcount)**2
            std = (std/(len(T)-1))**0.5/self.nshots
            
            print('The relative standard deviation of the measurement results is ... ' + str(round(len(T)*std*100)) + ' % ,')
            print('while the expected value for a uniform distribution is .......... ' + \
                  str(round(len(T)*((len(T)-1)/self.nshots/len(T)**2)**0.5*100))+ ' %')
            
            print('')
            print('Drawing the circuit...')
            circuit_drawer(circ,output='mpl',filename=self.filename)
            circuit_drawer(circ,output='latex_source',filename=self.filename+'.tex')
            print('A .png and latex source file with your circuit have been generated under your specified path/filename')

qT = quantumTransversal()
widgets.interact(qT.defineSys,symgroup='2x1')
widgets.interact(qT.permuteT,permuteT=False)
widgets.interact(qT.permutation,T_permutation='123')
widgets.interact(qT.isOnDemand,onDemandT=False)
widgets.interact(qT.onDemandT,onDemandT='000,001,011,111')
widgets.interact(qT.shotN,NoOfShots='1024')
widgets.interact(qT.setfilename,path_filename='output')

button = widgets.Button(description="Run")
display(button)

out = widgets.Output()
display(out)

button.on_click(qT.run)

interactive(children=(Text(value='2x1', description='symgroup'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Checkbox(value=False, description='permuteT'), Output()), _dom_classes=('widget-interact…

interactive(children=(Text(value='123', description='T_permutation'), Output()), _dom_classes=('widget-interac…

interactive(children=(Checkbox(value=False, description='onDemandT'), Output()), _dom_classes=('widget-interac…

interactive(children=(Text(value='000,001,011,111', description='onDemandT'), Output()), _dom_classes=('widget…

interactive(children=(Text(value='1024', description='NoOfShots'), Output()), _dom_classes=('widget-interact',…

interactive(children=(Text(value='output', description='path_filename'), Output()), _dom_classes=('widget-inte…

Button(description='Run', style=ButtonStyle())

Output()